In [2]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8'].reset_index(drop=True)

def get_first_level(x):
    tags = []
    for part in x.split(','):
        tags.append(part.split(':')[0].strip())
    return ', '.join(tags)

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)
df['second_level'] = df['tags'].apply(get_second_level)
df['third_level'] = df['tags'].apply(get_third_level)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import re
import pandas as pd

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

# Разделение классов в first_level на списки
df['first_level_list'] = df['first_level'].apply(lambda x: x.split(', '))

# Преобразование текстов меток в мультилейбел формат
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['first_level_list'])

# Очистка текстов
df['cleaned_title'] = df['title'].apply(clean_text)

# Разделение данных на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)

X_train = train_df['cleaned_title']
y_train = y[train_df.index]

X_valid = test_df['cleaned_title']
y_valid = y[test_df.index]

X_test = test_df['cleaned_title']
y_test = y[test_df.index]

# Применение TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=350,
    min_df=3,
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)
X_test_tfidf = tfidf.transform(X_test)

# Создание модели CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=2000, 
    learning_rate=0.03,
    depth=4,
    verbose=500,
    random_seed=42,
    eval_metric='AUC',
    early_stopping_rounds=2000,
)

# Обертка CatBoostClassifier в OneVsRestClassifier
ovr_model = OneVsRestClassifier(catboost_model)

# Обучение модели на преобразованных данных
ovr_model.fit(X_train_tfidf, y_train)

# Предсказание на тестовых данных
y_pred = ovr_model.predict(X_test_tfidf)

# Вычисление accuracy и F1-меры для мультилейбел данных
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro:.4f}')
print(f'F1 macro: {f1_macro:.4f}')

0:	total: 59.2ms	remaining: 1m 58s
500:	total: 452ms	remaining: 1.35s
1000:	total: 829ms	remaining: 827ms
1500:	total: 1.15s	remaining: 383ms
1999:	total: 1.47s	remaining: 0us
0:	total: 624us	remaining: 1.25s
500:	total: 370ms	remaining: 1.11s
1000:	total: 731ms	remaining: 730ms
1500:	total: 1.1s	remaining: 366ms
1999:	total: 1.47s	remaining: 0us
0:	total: 830us	remaining: 1.66s
500:	total: 371ms	remaining: 1.11s
1000:	total: 737ms	remaining: 735ms
1500:	total: 1.11s	remaining: 368ms
1999:	total: 1.48s	remaining: 0us
0:	total: 747us	remaining: 1.49s
500:	total: 368ms	remaining: 1.1s
1000:	total: 734ms	remaining: 733ms
1500:	total: 1.1s	remaining: 366ms
1999:	total: 1.45s	remaining: 0us
0:	total: 686us	remaining: 1.37s


/usr/local/anaconda3/envs/venv_mvp_matching/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(


500:	total: 371ms	remaining: 1.11s
1000:	total: 746ms	remaining: 745ms
1500:	total: 1.11s	remaining: 369ms
1999:	total: 1.45s	remaining: 0us
0:	total: 703us	remaining: 1.41s


/usr/local/anaconda3/envs/venv_mvp_matching/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(


500:	total: 370ms	remaining: 1.11s
1000:	total: 738ms	remaining: 737ms
1500:	total: 1.12s	remaining: 373ms
1999:	total: 1.5s	remaining: 0us
0:	total: 893us	remaining: 1.79s
500:	total: 377ms	remaining: 1.13s
1000:	total: 744ms	remaining: 742ms
1500:	total: 1.09s	remaining: 362ms
1999:	total: 1.41s	remaining: 0us
0:	total: 741us	remaining: 1.48s
500:	total: 374ms	remaining: 1.12s
1000:	total: 763ms	remaining: 761ms
1500:	total: 1.16s	remaining: 386ms
1999:	total: 1.54s	remaining: 0us
0:	total: 1.39ms	remaining: 2.77s
500:	total: 365ms	remaining: 1.09s
1000:	total: 731ms	remaining: 729ms
1500:	total: 1.04s	remaining: 346ms
1999:	total: 1.36s	remaining: 0us
0:	total: 1.48ms	remaining: 2.96s


...

/usr/local/anaconda3/envs/venv_mvp_matching/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 29 is present in all training examples.
  warnings.warn(


500:	total: 372ms	remaining: 1.11s
1000:	total: 764ms	remaining: 762ms
1500:	total: 1.14s	remaining: 378ms
1999:	total: 1.5s	remaining: 0us
0:	total: 1.28ms	remaining: 2.56s
500:	total: 378ms	remaining: 1.13s
1000:	total: 754ms	remaining: 753ms
1500:	total: 1.13s	remaining: 376ms
1999:	total: 1.5s	remaining: 0us
0:	total: 660us	remaining: 1.32s
500:	total: 385ms	remaining: 1.15s
1000:	total: 759ms	remaining: 757ms
1500:	total: 1.13s	remaining: 377ms
1999:	total: 1.51s	remaining: 0us
0:	total: 688us	remaining: 1.38s
500:	total: 375ms	remaining: 1.12s
1000:	total: 756ms	remaining: 755ms
1500:	total: 1.1s	remaining: 367ms
1999:	total: 1.44s	remaining: 0us
0:	total: 1.37ms	remaining: 2.74s
500:	total: 385ms	remaining: 1.15s
1000:	total: 765ms	remaining: 763ms
1500:	total: 1.14s	remaining: 380ms
1999:	total: 1.52s	remaining: 0us
0:	total: 727us	remaining: 1.45s
500:	total: 405ms	remaining: 1.21s
1000:	total: 793ms	remaining: 792ms
1500:	total: 1.17s	remaining: 390ms
1999:	total: 1.55s	remai

/usr/local/anaconda3/envs/venv_mvp_matching/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 37 is present in all training examples.
  warnings.warn(


500:	total: 386ms	remaining: 1.15s
1000:	total: 746ms	remaining: 745ms
1500:	total: 1.08s	remaining: 359ms
1999:	total: 1.42s	remaining: 0us
Accuracy: 0.5357
F1 micro: 0.7026
F1 macro: 0.3019


/usr/local/anaconda3/envs/venv_mvp_matching/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
y_pred_labels = mlb.inverse_transform(y_pred)
#y_pred_formatted = [", ".join(labels) for labels in y_pred_labels]

test_df = df.loc[test_df.index]
test_df['predicted_tags'] = y_pred_labels

In [2]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
# from catboost import CatBoostClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import accuracy_score, f1_score
# import re


# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
#     return text

# label_encoder = LabelEncoder()
# df['encoded_first_level'] = label_encoder.fit_transform(df['first_level'])
# class_counts = df['encoded_first_level'].value_counts()
# valid_classes = class_counts[class_counts >= 2].index


# df['text_for_classification'] = df['title']
# df['cleaned_title'] = df['text_for_classification'].apply(clean_text)

# # X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], df['encoded_first_level'], test_size=0.4, random_state=42)
# train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)

# X_train = train_df[train_df['encoded_first_level'].isin(valid_classes)]['cleaned_title']
# y_train = train_df[train_df['encoded_first_level'].isin(valid_classes)]['encoded_first_level']

# X_valid = test_df[test_df['encoded_first_level'].isin(valid_classes)]['cleaned_title']
# y_valid = test_df[test_df['encoded_first_level'].isin(valid_classes)]['encoded_first_level']

# X_test = test_df['cleaned_title']
# y_test = test_df['encoded_first_level']

# tfidf = TfidfVectorizer(
#     ngram_range=(1, 3),
#     max_features=350,
#     min_df=3,
# )
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_valid_tfidf = tfidf.transform(X_valid)
# X_test_tfidf = tfidf.transform(X_test)

# catboost_model = CatBoostClassifier(
#     iterations=3500, 
#     learning_rate=0.03,
#     depth=4,
#     verbose=100,
#     random_seed=42,
#     eval_metric='AUC',
#     early_stopping_rounds=2000,
# )
# catboost_model.fit(X_train_tfidf, y_train, eval_set=(X_valid_tfidf, y_valid))
# y_pred = catboost_model.predict(X_test_tfidf)

# accuracy = accuracy_score(y_test, y_pred)
# f1_micro = f1_score(y_test, y_pred, average='micro')
# f1_macro = f1_score(y_test, y_pred, average='macro')
# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1 micro: {f1_micro}')
# print(f'F1 macro: {f1_macro}')


# test_df = df.loc[y_test.index]
# test_df['predicted_tags'] = label_encoder.inverse_transform(y_pred).tolist()

# test_df['predicted_tags'] = test_df['predicted_tags'].apply(lambda x: [x])

# test_df

0:	test: 0.4479630	best: 0.4479630 (0)	total: 76ms	remaining: 4m 25s
100:	test: 0.7986136	best: 0.7986136 (100)	total: 1.26s	remaining: 42.6s
200:	test: 0.8144357	best: 0.8144357 (200)	total: 2.45s	remaining: 40.2s
300:	test: 0.8194408	best: 0.8197692 (279)	total: 3.65s	remaining: 38.8s
400:	test: 0.8218447	best: 0.8219183 (396)	total: 4.79s	remaining: 37s
500:	test: 0.8246211	best: 0.8255688 (491)	total: 6.02s	remaining: 36s
600:	test: 0.8261489	best: 0.8263597 (562)	total: 7.2s	remaining: 34.8s
700:	test: 0.8289275	best: 0.8289275 (700)	total: 8.4s	remaining: 33.6s
800:	test: 0.8303036	best: 0.8303055 (797)	total: 9.6s	remaining: 32.4s
900:	test: 0.8291147	best: 0.8304174 (805)	total: 10.8s	remaining: 31s
1000:	test: 0.8303214	best: 0.8304174 (805)	total: 11.9s	remaining: 29.8s
1100:	test: 0.8306391	best: 0.8307394 (1093)	total: 13.1s	remaining: 28.6s
1200:	test: 0.8298231	best: 0.8307489 (1112)	total: 14.3s	remaining: 27.3s
1300:	test: 0.8304961	best: 0.8307489 (1112)	total: 15.4s	r

In [7]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(df, pred_df):
    df["tags_new"] = df["tags"].apply(lambda l: l.split(', '))
    df["tags_split"] = df["tags_new"].apply(lambda l: split_tags(l))

    # pred_df["predicted_tags"] = pred_df["predicted_tags"].apply(ast.literal_eval)
    pred_df["predicted_tags_split"] = pred_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in df.iterrows():
        predicted_tags = pred_df[pred_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

sample_df = pd.read_csv('sample_submission.csv')
sample_df['predicted_tags'] = sample_df['predicted_tags'].apply(lambda x: [x])

final_score = find_iou_for_sample_submission(test_df, sample_df)
print(final_score)


final_score = find_iou_for_sample_submission(test_df, test_df)
print(final_score)

0.07057823129251697
0.3944444444444446


In [6]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from catboost import CatBoostClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.linear_model import LogisticRegression
# import re

# # Функция для очистки текста
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
#     return text

# # Применение очистки текста к колонке 'title'
# df['cleaned_title'] = df['title'].apply(clean_text)

# # Сплит меток по запятой и удаление лишних пробелов
# df['labels'] = df['first_level'].apply(lambda x: [label.strip() for label in x.split(',')])

# # Преобразование меток в формат, подходящий для многометочного обучения
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(df['labels'])

# X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], y, test_size=0.4, random_state=42)

# tfidf_params = {
#     # 'max_df': 0.9,
#     'min_df': 5,
#     'ngram_range': (1,2),
#     'max_features': 500
# }

# catboost_model = CatBoostClassifier(
#     iterations=1000, 
#     learning_rate=0.1,
#     depth=6,
#     verbose=False,
#     random_seed=42,
# )

# log_reg_model = LogisticRegression()


# # Создание пайплайна
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(**tfidf_params)),
#     ('catboost', OneVsRestClassifier(catboost_model))
# ])

# # Обучение модели
# pipeline.fit(X_train, y_train)

# # Предсказание на тестовой выборке
# y_pred = pipeline.predict(X_test)

# # Оценка точности и F1-меры
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='micro')
# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1 Score: {f1:.4f}')

# # Доступ к словарю tfidf и количество слов
# tfidf = pipeline.named_steps['tfidf']
# vocab_size = len(tfidf.vocabulary_)
# print(f'TFIDF Vocabulary Size: {vocab_size}')

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 11 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 36 is pre

Accuracy: 0.5452
F1 Score: 0.6641
TFIDF Vocabulary Size: 241


In [ ]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter


if __name__ == '__main__':

    try:
        pred_submission = sample_submission #pd.read_csv(pred_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке решения участника'
    try:
        true_submission = ground_truth #pd.read_csv(true_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке эталонного решения'


    final_score = find_iou_for_sample_submission(pred_submission, true_submission)
    print("FINAL_SCORE", 'число от 0 до 1') #final_score)

In [125]:
for x,y in df['tags'].value_counts().to_dict().items():
    print(x, y)

Массовая культура: Юмор и сатира 224
Религия и духовность: астрология 57
Еда и напитки: Кулинария 41
Массовая культура, Карьера 37
Массовая культура 24
Путешествия, События и достопримечательности: Исторические места и достопримечательности 22
Массовая культура: Юмор и сатира, Семья и отношения 22
Спорт: Рыбалка 20
Дом и сад: Дизайн интерьера 20
Массовая культура, Музыка и аудио 18
Транспорт, Спорт: Автогонки, События и достопримечательности: Спортивные события 17
Еда и напитки: Кулинария, Массовая культура 16
Массовая культура, Спорт 15
Музыка и аудио: Комедия и стендап (Музыка и аудио), События и достопримечательности: Концерты и музыкальные мероприятия 14
Массовая культура: Отношения знаменитостей 13
Религия и духовность: астрология, События и достопримечательности: Комедия и стендап 12
Массовая культура, Хобби и интересы: Декоративно-прикладное искусство 11
Семья и отношения 11
Фильмы и анимация: Семейные и детские фильмы, Фильмы и анимация: Фильмы и анимация  11
Массовая культура,

In [121]:
df[df['first_level'] == 'Массовая культура, Карьера']

,video_id,title,description,tags,first_level,second_level,third_level,cleaned_title,text_for_classification
91,51304cc9ff8e942ce481282b6952dd86,Артмеханика. Концерт группы TIHOTIHO,Концерт группы TIHOTIHO,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,концерт группы tihotiho,Концерт группы TIHOTIHO
196,02b797508b9ffc941887703dfb17d365,Артмеханика. Сезон 2. Выпуск 6. Эволюция веду...,Влад Маленко и Ольга Ершова разбираются в том...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,влад маленко и ольга ершова разбираются в том...,Влад Маленко и Ольга Ершова разбираются в том...
220,034753227cf7772245e3fa97c5274932,"Артмеханика. Игра ""Лицо рекламы"".","Игра ""Лицо рекламы""","Массовая культура, Карьера","Массовая культура, Карьера",-,-,игра лицо рекламы,"Игра ""Лицо рекламы"""
234,13aba71d6501df8db686919affbc9b72,Артмеханика. Интервью. Денис Рогов.,Что такое метавселенная? Где мы можем ощутить ...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что такое метавселенная где мы можем ощутить к...,Что такое метавселенная? Где мы можем ощутить ...
236,63d9eab25494382081b6ad100efa7ad4,Артмеханика. Концерт группы Dabro.,Все самые лучшие хиты группы Dabro.,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,все самые лучшие хиты группы dabro,Все самые лучшие хиты группы Dabro.
258,d42d194ba6c4e674a6a625106a76fb1e,Артмеханика. Интервью. Степан Липгарт.,Что собой представляет современная архитектура...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что собой представляет современная архитектура...,Что собой представляет современная архитектура...
259,343470324af6902808606cf1a7503575,Артмеханика. Интервью с сёстрами Набока.,Как сёстры Набока покорили известных актёров и...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,как сёстры набока покорили известных актёров и...,Как сёстры Набока покорили известных актёров и...
261,7413b5707b2119fc392f0ac00cc2a8a1,Артмеханика. Интервью с Анастасией Моргун и Юр...,Зачем сериал Санта-Барбара вернулся к нам спус...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,зачем сериал сантабарбара вернулся к нам спуст...,Зачем сериал Санта-Барбара вернулся к нам спус...
265,9444666b1f0d17c502e46ecbaad08b67,Артмеханика. Сезон 2. Выпуск 15. Подлинное иск...,В нашей студии стало жарко! “Арт Механика” реш...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,в нашей студии стало жарко арт механика решила...,В нашей студии стало жарко! “Арт Механика” реш...
267,c477b6170b5991c1550032c394d93fd0,Артмеханика. Интервью с Евгением Чесом.,Куда приведет тенденция к тотальной кастомизац...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,куда приведет тенденция к тотальной кастомизац...,Куда приведет тенденция к тотальной кастомизац...


In [12]:
from sklearn.metrics import classification_report

print(classification_report(df['first_level'], df_pred['first_level']))

                                                                                       precision    recall  f1-score   support

                                                                    , Стиль и красота       0.00      0.00      0.00         1
                                                                     Бизнес и финансы       0.00      0.00      0.00         6
                                                                            Дом и сад       0.80      0.18      0.30        22
                                                                        Еда и напитки       0.50      0.56      0.53        64
                                                                             Животные       0.00      0.00      0.00         0
                                                                 Здоровый образ жизни       0.00      0.00      0.00         1
                                                                                 Игры       0.12      0.33    

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca